# CLAMPS Lidar Vertical Stare Tutorial

This tutorial will show you how to work with vertical stare data from the Halo Photonics Streamline lidars with the CLAMPS systems at OU and NSSL.


## Getting Data
CLAMPS data is stored on a THREDDS server (https://data.nssl.noaa.gov/thredds/catalog/FRDD/CLAMPS/catalog.html). We are going to use Siphon to grab the data from the server. If you use Anaconda, Siphon can be download by running
```
conda install -c conda-forge siphon
```
on the command line. If you use pip the command would be
```
pip install siphon
```

## Vertical Stares
While a vertical stare may be simple, it can provide valuable information. These stares can observe subtle vertical motion associated with features such as gravity waves and bores or when run constantly over a long enough period of time they can be used to retrieve profiles of vertical velocity variance.'

We are going to work with vertical stare data from the CLAMPS 1 lidar during the BLISS-FUL campaign on June 20, 2021. Let's first get the data:

In [ ]:
from siphon.catalog import TDSCatalog
from datetime import datetime
import numpy as np

# Catalog for the CLAMPS1 ingested Stare data
catURL = "https://data.nssl.noaa.gov/thredds/catalog/FRDD/CLAMPS/clamps/clamps1/ingested/clampsdlfpC1.b1/catalog.xml"

# Open the catalog
cat = TDSCatalog(catURL)

# Date we want to grab
dt = datetime(2021, 6, 20)

# Get the dates for all the netCDF datasets (have to do it this way because file names can be messed up)
nc_dates = []
for ds in cat.datasets:
    if '.cdf' in ds:
        try:
            nc_dates.append(datetime.strptime(ds, "clampsdlfpC1.b1.%Y%m%d.%H%M%S.cdf"))
        except:
            nc_dates.append(datetime(2100,1,1))
            print('This file name is messed up: ' + ds)

nc_dates = np.array(nc_dates)

# Find the index of the date we want
ind = np.argmin(np.abs(dt - nc_dates))

# Get the dataset
ds = cat.datasets[ind]

# Download the dataset we identified to our current working directory
ds.download()

The data is now downloaded to your working directory. It is a netCDF file so we are going to open it with the netCDF4 package and look at the variables in the netCDF file

In [ ]:
from netCDF4 import Dataset

nc = Dataset(ds,'r')

print(nc)

Let's read in the velocity, backscatter, intensity, cbh, height, and hour fields.

In [ ]:
hour = nc.variables['hour'][:]
bsc = nc.variables['backscatter'][:]
intensity = nc.variables['intensity'][:]
velocity = nc.variables['velocity'][:]
cbh = nc.variables['cbh'][:]
height = nc.variables['height'][:]
nc.close()

Now we are going to remove data below a set intensity value and then plot the backscatter, cloud based height, and velocity fields in time-height plots.

In [ ]:
import matplotlib.pyplot as plt
import cmocean

# Use the intensity to threshold the data
foo = np.where(intensity < 1.007)

bsc[foo] = np.nan
velocity[foo] = np.nan

# We need to calculate height from the range values

fig, (backscatter, vert) =  plt.subplots(2, sharex=True)
fig.set_figheight(10)
fig.set_figwidth(15)

# There are so many points in this plot that it will wipe out your RAM if you try to plot all of it
# so we are only plotting up to 2 km

foo = np.where(height <= 2)[0]

a = backscatter.pcolormesh(hour,height[foo],np.log10(bsc[:,foo].T),cmap ='turbo', vmin=-7, vmax=-3, shading = 'auto')
backscatter.scatter(hour,cbh,color='k')
b = vert.pcolormesh(hour,height[foo],velocity[:,foo].T,cmap = 'seismic', vmin=-3, vmax=3, shading = 'auto')

cb = plt.colorbar(a, ax=backscatter)
cb.set_label('[1/ms]')

cb = plt.colorbar(b, ax=vert)
cb.set_label('[m/s]')
    
backscatter.set_ylim([0,2])
vert.set_ylim([0,2])

backscatter.set_ylabel('Height [km]')
vert.set_ylabel('Height [km]')
vert.set_xlabel('Hour [UTC]')

backscatter.set_title('Backscatter')
vert.set_title('Vertical Velocity')
plt.tight_layout()
plt.show()

Lets calculate 15 minute vertical velocity variance from this data

In [ ]:
avg_time = np.arange(7.5,1441,15)/60
w_var = np.ones((len(avg_time),len(height)))*np.nan

# Loop over avg_time can calculate the variance in that 30 minute period
for i in range(len(avg_time)):
    foo = np.where(((hour >= avg_time[i]-0.25) & (hour < avg_time[i]+0.25)))[0]
    w_var[i,:] = np.nanvar(velocity[foo,:],axis=0)

In [ ]:
foo = np.where(height <= 2)[0]

# And now plot that vertical velocity variance
fig, (var_plot) =  plt.subplots(1)
fig.set_figheight(5)
fig.set_figwidth(15)

a = var_plot.pcolormesh(avg_time,height[foo], w_var[:,foo].T,cmap ='viridis',vmin=0, vmax=2, shading = 'auto')

cb = plt.colorbar(a, ax=var_plot)
cb.set_label('[$m^2/s^2$]')

var_plot.set_ylim([0,2])

var_plot.set_ylabel('Height [km]')
var_plot.set_xlabel('Hour [UTC]')

var_plot.set_title('Vertical Velocity Variance')
plt.tight_layout()
plt.show()


Note: this vertical velocity variance calculation is fine when just trying to get an idea for what is happening for a particular day. If you want to produce publication quality vertical velocity variance you will have to use do a Lenshow correction to the data to remove noise from the lidar from the the variance. This correction is a little beyond this basic lidar training, but I can show you how to do it if you need it. My hot take on this topic is that lidar technology has advanced significantly is the 22 years since the Lenshow paper was written and the noise in the measurements is much less than before, making the correction less important to do. This is especially true when you are sampling with a large number of rays.